# Class 8 - 13.5.19

# `pandas`, `xarray` and Modeling

## String DataFrames

The raw data we have can sometimes come in the form of strings. Pandas support string DataFrames and allows for manipulation of these objects.

Note: I'm currently not discussing "bioinformatic sequences" of `ATGC` - these have a special `Seq` class in the Biopython package, the workhorse of Python bioinformatics.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

df_str = pd.DataFrame(['a', 'b', 'c', 'D  ', 'EFG', 'hij', 'k', '', ' LMNOP', 'QRST', 'UV', ' wxy', 'z'],
                      columns=['alpha'])
df_str

matplotlib data path C:\Users\Hagai\Miniconda3\envs\general\lib\site-packages\matplotlib\mpl-data
loaded rc file C:\Users\Hagai\.matplotlib\matplotlibrc
matplotlib version 2.1.2
verbose.level helpful
interactive is False
platform is win32
$HOME=C:\Users\Hagai
CACHEDIR=C:\Users\Hagai\.matplotlib
Using fontManager instance from C:\Users\Hagai\.matplotlib\fontList.json
backend module://ipykernel.pylab.backend_inline version unknown
backend module://ipykernel.pylab.backend_inline version unknown


alpha
0        a
1        b
2        c
3      D  
4      EFG
5      hij
6        k
7         
8    LMNOP
9     QRST
10      UV
11     wxy
12       z

backend module://ipykernel.pylab.backend_inline version unknown


To access the varienty of string methods in pandas, use the `.str` attribute of a Series.

In [2]:
df_str.alpha.str.lower()

0          a
1          b
2          c
3        d  
4        efg
5        hij
6          k
7           
8      lmnop
9       qrst
10        uv
11       wxy
12         z
Name: alpha, dtype: object

In [3]:
df_str.alpha.str.strip()  # remove whitespaces

0         a
1         b
2         c
3         D
4       EFG
5       hij
6         k
7          
8     LMNOP
9      QRST
10       UV
11      wxy
12        z
Name: alpha, dtype: object

When parsing `.csv` data, the column names can sometimes be unorderly, having whitespaces and different capitalization conventions. Here's an example of how to make them look alike:

In [4]:
# Assume we parsed this DataFrame from some third-party source
csv_df = pd.DataFrame([], columns=[' Name', 'Age ', ' Gender ', 'Is single', 'Has children', 'Num of Children'])
csv_df

Empty DataFrame
Columns: [ Name, Age ,  Gender , Is single, Has children, Num of Children]
Index: []

Due to the fact that a column of a DataFrame is a regular Index object, we can use our string methods to make it look better:

In [5]:
csv_df.columns = csv_df.columns.str.strip()\
                               .str.lower()\
                               .str.replace(' ', '_')
csv_df  # stripped of leading and trailing whitespaces, 
        # removed capital letters
        # and replaced spaces with underscores

Empty DataFrame
Columns: [name, age, gender, is_single, has_children, num_of_children]
Index: []

We can get the first letter in each entry:

In [6]:
df_str.alpha.str.get(0)

0       a
1       b
2       c
3       D
4       E
5       h
6       k
7     NaN
8        
9       Q
10      U
11       
12      z
Name: alpha, dtype: object

You can also use regular expressions to extract patterns from each row of the Series.

In the mean time, a summary of all the string-related methods in pandas can be found [here](http://pandas.pydata.org/pandas-docs/stable/text.html). They contain most of the Pythonic string-related operations.

Note: String operations aren't as fast as numerical computations since they're currently implemented mostly with Python in the backend, and not C.

## Pandas Concatenation

Similar to numpy arrays, Series and DataFrame objects can be concatenated as well. The indexes of the pandas objects is what usually makes this operation non-trivial.

While this discussion might not seem to you super important, it's actually more useful than you'd think. Thinking of our different sets of data in terms of a large database can sometimes be a good analogy.

A full tutorial on this manner can be found in the official pandas tutorial, right [here](https://pandas.pydata.org/pandas-docs/stable/merging.html).

In [7]:
ser1 = pd.Series(['a', 'b', 'c'], index=[1, 2, 3])
ser2 = pd.Series(['d', 'e', 'f'], index=[4, 5, 6])
pd.concat([ser1, ser2])  # row-wise (axis=0) by default

1    a
2    b
3    c
4    d
5    e
6    f
dtype: object

In [8]:
# This simple behavior can be achieved using the append method
ser1.append(ser2)

1    a
2    b
3    c
4    d
5    e
6    f
dtype: object

In [9]:
df1 = pd.DataFrame([['a', 'A'], ['b', 'B']], columns=['let', 'LET'], index=[0, 1])
df2 = pd.DataFrame([['c', 'C'], ['d', 'D']], columns=['let', 'LET'], index=[2, 3])
pd.concat([df1, df2])  # again, along the first axis

let LET
0   a   A
1   b   B
2   c   C
3   d   D

In [10]:
df1 = pd.DataFrame([['a', 'A'], ['b', 'B']], columns=['let1', 'LET1'], index=[0, 1])
df2 = pd.DataFrame([['c', 'C'], ['d', 'D']], columns=['let2', 'LET2'], index=[2, 3])
pd.concat([df1, df2])  # pandas can't make the column index compatible, so it resorts to columnar concat
# the same result is achieved by pd.concat([df1, df2], axis=1)

LET1 LET2 let1 let2
0    A  NaN    a  NaN
1    B  NaN    b  NaN
2  NaN    C  NaN    c
3  NaN    D  NaN    d

If we wish to keep the integrity of the indices, we can use the `verify_integrity` keyword:

In [11]:
df1 = pd.DataFrame([['a', 'A'], ['b', 'B']], columns=['let', 'LET'], index=[0, 1])
df2 = pd.DataFrame([['c', 'C'], ['d', 'D']], columns=['let', 'LET'], index=[0, 2])
pd.concat([df1, df2], verify_integrity=True)

ValueError: Indexes have overlapping values: [0]

If we don't care about the indices, we can just ignore them:

In [12]:
pd.concat([df1, df2], ignore_index=True)  # creates a new index

let LET
0   a   A
1   b   B
2   c   C
3   d   D

We can also create a new MultiIndex if it makes more sense:

In [13]:
pd.concat([df1, df2], keys=['df1', 'df2'])  # "remembers" the origin of the data, super useful!

let LET
df1 0   a   A
    1   b   B
df2 0   c   C
    2   d   D

A real world example of concatenation is when we wish to join two datasets taken at different times. For example, if we conducted in day 1 measurements at times 8:00, 10:00, 14:00 and 16:00, but during day 2 we were a bit dizzy, and conducted the measurements at 8:00, 10:00, 13:00 and 16:30. On top of that, we recorded another parameter that we forget to measure at day 1.

The default concatenation behavior of pandas keeps all the data. In database terms (SQL people rejoice!) it's called an "outer join":

In [14]:
# Prepare mock data
times_day1 = pd.to_datetime(['08:00', '10:00', '14:00', '16:00'], format='%H:%M')
times_day2 = pd.to_datetime(['08:00', '10:00', '13:00', '16:30'], format='%H:%M')

temp_day1 = [36.6, 36.7, 37.0, 36.8]
temp_day2 = [35.9, 36.1, 36.5, 36.2]

humid_day1 = [30., 31., 30.4, 30.4]
humid_day2 = [32.2, 34.2, 30.9, 32.6]

light_day2 = [200, 130, 240, 210]

data_day1 = pd.DataFrame({'temp': temp_day1, 'humid': humid_day1}, index=times_day1)
data_day2 = pd.DataFrame({'temp': temp_day2, 'humid': humid_day2,
                          'light': light_day2}, index=times_day2)
data_day1

humid  temp
1900-01-01 08:00:00   30.0  36.6
1900-01-01 10:00:00   31.0  36.7
1900-01-01 14:00:00   30.4  37.0
1900-01-01 16:00:00   30.4  36.8

In [15]:
data_day2

humid  light  temp
1900-01-01 08:00:00   32.2    200  35.9
1900-01-01 10:00:00   34.2    130  36.1
1900-01-01 13:00:00   30.9    240  36.5
1900-01-01 16:30:00   32.6    210  36.2

In [16]:
# Outer join
pd.concat([data_day1, data_day2], join='outer')  # outer join is the default behavior  

humid  light  temp
1900-01-01 08:00:00   30.0    NaN  36.6
1900-01-01 10:00:00   31.0    NaN  36.7
1900-01-01 14:00:00   30.4    NaN  37.0
1900-01-01 16:00:00   30.4    NaN  36.8
1900-01-01 08:00:00   32.2  200.0  35.9
1900-01-01 10:00:00   34.2  130.0  36.1
1900-01-01 13:00:00   30.9  240.0  36.5
1900-01-01 16:30:00   32.6  210.0  36.2

To take the intersection of the columns we have to use inner join. The intersection is all the columns that are common in all datasets.

In [17]:
# Inner join - the excess data column was dropped (index is still not unique)
pd.concat([data_day1, data_day2], join='inner')

humid  temp
1900-01-01 08:00:00   30.0  36.6
1900-01-01 10:00:00   31.0  36.7
1900-01-01 14:00:00   30.4  37.0
1900-01-01 16:00:00   30.4  36.8
1900-01-01 08:00:00   32.2  35.9
1900-01-01 10:00:00   34.2  36.1
1900-01-01 13:00:00   30.9  36.5
1900-01-01 16:30:00   32.6  36.2

One can also specify the exact columns that should be the result of the join operation using the `columns` keyword. All in all, this basic functionality is easy to understand and allows for high flexibility. If you wish to conduct a more sophisticated join operation, you should use `pd.merge()`, as we'll show below.

Finally, joining on the columns will require the indices to be unique:

In [18]:
pd.concat([data_day1, data_day2], join='inner', axis='columns')

humid  temp  humid  light  temp
1900-01-01 08:00:00   30.0  36.6   32.2    200  35.9
1900-01-01 10:00:00   31.0  36.7   34.2    130  36.1

This doesn't look so good. The columns are a mess and we're barely left with any data. The right operation for this job is a `pd.merge`.

Merging two DataFrames into one is done in the following manner:

In [19]:
times = pd.to_datetime(['08:00', '10:00', '14:00', '16:00'], format='%H:%M')

temp_day1 = [36.6, 36.7, 37.0, 36.8]
humid_day1 = [30., 31., 30.4, 30.4]

df_temp = pd.DataFrame({'temp': temp_day1}, index=times)
df_humid = pd.DataFrame({'humid': humid_day1}, index=times)

pd.merge(df_temp, df_humid, right_index=True, left_index=True)  # merge with indices

temp  humid
1900-01-01 08:00:00  36.6   30.0
1900-01-01 10:00:00  36.7   31.0
1900-01-01 14:00:00  37.0   30.4
1900-01-01 16:00:00  36.8   30.4

In [20]:
# Slightly different data example
temp_day1 = [36.6, 36.7, 37.0, 36.8]
humid_day1 = [30., 31., 30.4, 30.4]

temp_day2 = [37.1, 36.6, 36.6, 36.7]

df_day1 = pd.DataFrame({'temp': temp_day1, 'humid': humid_day1}, index=times)
df_day2 = pd.DataFrame({'temp': temp_day2}, index=times)

df_day1

humid  temp
1900-01-01 08:00:00   30.0  36.6
1900-01-01 10:00:00   31.0  36.7
1900-01-01 14:00:00   30.4  37.0
1900-01-01 16:00:00   30.4  36.8

In [21]:
# Merging (as a method, not a procedure)
df_day1.merge(df_day2, left_index=True, right_index=True)
# This merge "remembers" the origin of the data, the "suffixes" keyword allows us to determine the new suffix

humid  temp_x  temp_y
1900-01-01 08:00:00   30.0    36.6    37.1
1900-01-01 10:00:00   31.0    36.7    36.6
1900-01-01 14:00:00   30.4    37.0    36.6
1900-01-01 16:00:00   30.4    36.8    36.7

In [22]:
# One more example, more similar to traditional databases
mouse_id = [511, 512, 513, 514]
meas1 = [67, 66, 89, 92]
meas2 = [45, 45, 65, 61]

data1 = pd.DataFrame({'id': mouse_id, 'blood_vol': meas1})
data2 = pd.DataFrame({'id': mouse_id, 'monocytes': meas2})
data1

blood_vol   id
0         67  511
1         66  512
2         89  513
3         92  514

In [23]:
data1.merge(data2)  # merge identified that the only "key" connecting the two tables was the 'id' key

blood_vol   id  monocytes
0         67  511         45
1         66  512         45
2         89  513         65
3         92  514         61

Database-like operation is a very broad topic in pandas, and it's another great example of why should you keep your data in a DataFrame.

If you feel like this join and merge operation might help you, don't hesitate to consult the official documents on this topic, linked [here](https://pandas.pydata.org/pandas-docs/stable/merging.html) (again).

## GroupBy

Yet another SQL-like feature that pandas posses is the group-by operation, sometimes known as "split-apply-combine".

In [24]:
# Mock data
subject = [100, 101, 102, 106, 108, 109]
dead = [True, True, False, True, False, False]
took_med = [False, True, True, False, False, True]
val1 = np.random.random(6)
val2 = np.random.random(6)
df = pd.DataFrame({'dead': dead, 'took_med': took_med,
                   'val1': val1, 'val2': val2}, index=subject)
df

dead  took_med      val1      val2
100   True     False  0.467618  0.647136
101   True      True  0.777130  0.082121
102  False      True  0.949176  0.974665
106   True     False  0.127160  0.332520
108  False     False  0.267529  0.288290
109  False      True  0.220782  0.004586

The most sensible thing to do is to group by either the "dead" or the "took_med" column (or both). This is the "split" part.

In [25]:
grouped = df.groupby('dead')
grouped  # DataFrameGroupBy object - intermediate object ready to be evaluated

This intermediate object is an internal pandas representation which should allow it to run very fast computation the moment we want to actually _know_ something about these groups. Assuming we want the mean of `val1`, as long as we won't specifically write `grouped.mean()` pandas will do very little in terms of actual computation. It's called "lazy evaluation".

The intermediate object has some useful attributes:

In [26]:
grouped.groups

{False: Int64Index([102, 108, 109], dtype='int64'),
 True: Int64Index([100, 101, 106], dtype='int64')}

In [27]:
len(grouped)  # True and False

2

If we wish to run some actual processing, we have to use an aggregation function:

In [28]:
grouped.sum()

took_med      val1      val2
dead                               
False       2.0  1.437487  1.267541
True        1.0  1.371908  1.061777

In [29]:
grouped.mean()

took_med      val1      val2
dead                               
False  0.666667  0.479162  0.422514
True   0.333333  0.457303  0.353926

In [30]:
grouped.size()

dead
False    3
True     3
dtype: int64

If we just wish to see one of the groups, we can use `get_group()`:

In [31]:
grouped.get_group(True)

took_med      val1      val2
100     False  0.467618  0.647136
101      True  0.777130  0.082121
106     False  0.127160  0.332520

We can also call several functions at once using the `.agg` attribute:

In [32]:
grouped.agg([np.sum, np.var])

took_med                val1                val2          
           sum       var       sum       var       sum       var
dead                                                            
False      2.0  0.333333  1.437487  0.166231  1.267541  0.248776
True       1.0  0.333333  1.371908  0.105695  1.061777  0.080154

GroupBy offers many more features, documented [online](http://pandas.pydata.org/pandas-docs/stable/groupby.html).

## Exercise

1. _Strings in pandas:_ 
    - Generate a 1000-length `pd.DataFrame` filled with 3-letter strings. Use the `string` module, and others, to generate it quickly.
    - Add a column indicating if the string in this row has a `z` in its 2nd character.
    - Add a third column swapping the case of the 3-letter string in these specific lines (`azI` to `AZi`). In the other lines it should remain uncapitalized.

2. _Concatenation:_
    - Create three DataFrames with random values and shapes of (10x2), (10x1), (15x3). Their index should be simple ordinal integers, and their column names should differ.
    - Concatenate these DFs over the second axis using `pd.concat`.
    - Concatenate these DFs over the second axis using `pd.merge`.

3. _GroupBy:_
    - Create a DataFrame with two columns, 10k entries in length. The first should be a "flag" column, having a value of either 0 or 1, randomly. The second should be A sine wave from 0 to 20$\pi$, length of 10k. This simulates measuring a parameter from two distinct groups.
    - GroupBy the DataFrame, creating a GroupBy object.
    - Plot and overlay the two traces (of the two flags).
    - Use the `.rolling()` method to create a rolling average window of length 5 and overlay the result.


## Exercise solutions below...

In [33]:
# 1
import string
import numpy as np
import pandas as pd

# Generate the array
letters = list(string.ascii_lowercase)
size = 1000
num_of_letters = 3 
chosen = np.random.choice(letters, size*num_of_letters)
chosen = chosen.reshape((size, num_of_letters))
df = pd.DataFrame(chosen, columns=['a', 'b', 'c'])
df['letters'] = df.a.str.cat(df.b.str.cat(df.c))

# Filter
char = 'z'
df['no_z'] = df['letters'].str.find(char) != 1
df['swap'] = df['letters'].where(df['no_z'], other=df['letters'].str.upper())
df.head(20)

a  b  c letters   no_z swap
0   l  m  a     lma   True  lma
1   f  w  t     fwt   True  fwt
2   n  q  b     nqb   True  nqb
3   q  a  d     qad   True  qad
4   f  h  g     fhg   True  fhg
5   a  t  s     ats   True  ats
6   v  i  w     viw   True  viw
7   n  i  a     nia   True  nia
8   r  p  y     rpy   True  rpy
9   t  q  v     tqv   True  tqv
10  g  w  g     gwg   True  gwg
11  r  n  y     rny   True  rny
12  b  l  z     blz   True  blz
13  z  i  e     zie   True  zie
14  k  f  u     kfu   True  kfu
15  x  l  l     xll   True  xll
16  e  i  p     eip   True  eip
17  p  d  v     pdv   True  pdv
18  i  p  u     ipu   True  ipu
19  h  z  w     hzw  False  HZW

In [34]:
# 2
dfa = pd.DataFrame(np.random.random((10, 2)), columns=['a', 'b'])
dfb = pd.DataFrame(np.random.random((10, 1)), columns=['c'])
dfc = pd.DataFrame(np.random.random((15, 3)), columns=['d', 'e', 'f'])

# Using concat
print(pd.concat([dfa, dfb, dfc], axis=1))
# Using merge
pd.merge(pd.merge(dfa, dfb, how='outer', left_index=True, right_index=True), 
         dfc, how='outer', left_index=True, right_index=True)

           a         b         c         d         e         f
0   0.721960  0.479197  0.099444  0.553364  0.035874  0.382604
1   0.948188  0.692544  0.387400  0.318574  0.697947  0.174908
2   0.132986  0.358062  0.065429  0.285649  0.099259  0.795915
3   0.344272  0.141087  0.187466  0.040071  0.888251  0.373946
4   0.942172  0.354470  0.529759  0.904777  0.830590  0.025147
5   0.531707  0.633662  0.325085  0.888547  0.203152  0.772623
6   0.461339  0.613543  0.748742  0.944104  0.393753  0.927445
7   0.146214  0.443801  0.346715  0.311424  0.282527  0.319768
8   0.386710  0.249720  0.464344  0.165226  0.245146  0.605569
9   0.508686  0.267013  0.123188  0.949701  0.145493  0.146770
10       NaN       NaN       NaN  0.568570  0.579946  0.272544
11       NaN       NaN       NaN  0.571234  0.146229  0.944688
12       NaN       NaN       NaN  0.611672  0.852926  0.734596
13       NaN       NaN       NaN  0.894587  0.936936  0.744179
14       NaN       NaN       NaN  0.085044  0.004389  0

a         b         c         d         e         f
0   0.721960  0.479197  0.099444  0.553364  0.035874  0.382604
1   0.948188  0.692544  0.387400  0.318574  0.697947  0.174908
2   0.132986  0.358062  0.065429  0.285649  0.099259  0.795915
3   0.344272  0.141087  0.187466  0.040071  0.888251  0.373946
4   0.942172  0.354470  0.529759  0.904777  0.830590  0.025147
5   0.531707  0.633662  0.325085  0.888547  0.203152  0.772623
6   0.461339  0.613543  0.748742  0.944104  0.393753  0.927445
7   0.146214  0.443801  0.346715  0.311424  0.282527  0.319768
8   0.386710  0.249720  0.464344  0.165226  0.245146  0.605569
9   0.508686  0.267013  0.123188  0.949701  0.145493  0.146770
10       NaN       NaN       NaN  0.568570  0.579946  0.272544
11       NaN       NaN       NaN  0.571234  0.146229  0.944688
12       NaN       NaN       NaN  0.611672  0.852926  0.734596
13       NaN       NaN       NaN  0.894587  0.936936  0.744179
14       NaN       NaN       NaN  0.085044  0.004389  0.385534

In [35]:
# 3
vals = np.array([0, 1])
needed = 100
stop = 20 * np.pi
chosen = np.random.choice(vals, needed)
df = pd.DataFrame({'flag': chosen, 'val': np.sin(np.linspace(start=0, stop=stop, num=needed))})

window_size = 5
grouped = df.groupby('flag')
grouped.val.plot()

df.val.rolling(window=window_size).mean().plot()

findfont: Matching :family=sans-serif:style=normal:variant=normal:weight=normal:stretch=normal:size=10.0 to DejaVu Sans ('C:\\Users\\Hagai\\Miniconda3\\envs\\general\\lib\\site-packages\\matplotlib\\mpl-data\\fonts\\ttf\\DejaVuSans.ttf') with score of 1.050000


## IntervalIndex

Pandas has a __TON__ of features and small implementation details that are there to make your life simpler. A small example, just to make you look for these deep in the documentation, is an `IntervalIndex`.

`IntervalIndex` is a type of index, just like a `DatetimeIndex` that we've seen many times, or a simple integer index. This index is special because it contain a start and a stop "time", and values can be included or excluded in this range.

In [36]:
df = pd.DataFrame([0, 1, 2, 3], index=pd.IntervalIndex.from_breaks((0, 1, 2, 3, 4)),
                  columns=['data'])
df  # notice the index

data
(0, 1]     0
(1, 2]     1
(2, 3]     2
(3, 4]     3

In [37]:
# Select a label on the edge of the index
df.loc[2]  # label-based, i.e. index-based and not value-based

data    1
Name: (1, 2], dtype: int64

In [38]:
df.loc[[1, 2]]

data
(0, 1]     0
(1, 2]     1

In [39]:
# Select a label contained in the index
df.loc[1.5]

data    1
Name: (1, 2], dtype: int64

There are types of datasets that "demand" an `IntervalIndex`, 
in the sense that it's very natural to use them to represent the data. 
For example, if you sample data from an electrode every microsecond, 
then the measurement at time `t` represents the values in times `(t-us, t]`.

# Higher Dimensionality - `xarray`

Pandas is amazing, but has its limits. A `DataFrame` can be a multi-dimensional container when using a `MultiIndex`, but it's limited to a subset of uses in which another layer of indexing makes sense.

In many occasions, however, our data is truly high-dimensional. A simple case could be electrophysiological recordings, or calcium traces. In these cases we have several indices (some can be categorical), like "Sex", "AnimalID", "Date", "TypeOfExperiment" and perhaps a few more. But the data itself is a vector of numbers representing voltage or fluorescence. Having this data in a DataFrame seems a bit "off" - what are the columns on this DF? Is each column a voltage measurement? Or if each column is a measurement, how do you deal with the indices? We can use nested columns (`MultiIndex` the columns), but it's not a very modular approach.

This is a classic example where pandas DF "fail", and indeed pandas used to have a higher-dimensionality container named `Panel`. However, late 2016 pandas devs deprecated it, publicly announcing that they intend to drop support for `Panel`s sometime in the future, and whoever needs a higher-dimensionality container should use `xarray`.

Thus, in essence `xarray` is a labeled n-dimensional array. Just like a DataFrame is a labeled 2D array, i.e. with names to its axes rather than numbers, in `xarray` each dimension has a name (`time`, `temp`, `voltage`) and its indices ("coordinates") can also have labels (like a timestamp, for example). In addition, each `xarray` object also has metadata attached to it, in which we can write details that do not fit a columnar structure (experimenter name, hardware and software used for acquisition, etc.).

## DataArray

In [40]:
import numpy as np
import xarray as xr


da = xr.DataArray(np.random.random((10, 2)))
da

<xarray.DataArray (dim_0: 10, dim_1: 2)>
array([[0.901048, 0.236656],
       [0.68286 , 0.123771],
       [0.311127, 0.195564],
       [0.272591, 0.661046],
       [0.751   , 0.566275],
       [0.085168, 0.153432],
       [0.417684, 0.720841],
       [0.681176, 0.56612 ],
       [0.671543, 0.7432  ],
       [0.89269 , 0.69314 ]])
Dimensions without coordinates: dim_0, dim_1

The basic building block of `xarray` is a DataArray, an n-dimensional counter part of a pandas' Series. It has two dimensions, just like the numpy array that its based upon. We didn't specify names for these dimensions, so currently they're called `dim_0` and `dim_1`. We also didn't specify coordinates (indices), so the printout doesn't report of any coordinates for the data.

In [41]:
da.values  # just like pandas

array([[0.90104791, 0.23665646],
       [0.6828599 , 0.12377064],
       [0.31112689, 0.19556449],
       [0.27259109, 0.66104582],
       [0.75100041, 0.56627539],
       [0.08516836, 0.15343153],
       [0.41768401, 0.72084051],
       [0.68117566, 0.5661196 ],
       [0.67154252, 0.74320025],
       [0.89268968, 0.69313964]])

In [42]:
da.coords

Coordinates:
    *empty*

In [43]:
da.dims

('dim_0', 'dim_1')

In [44]:
da.attrs

OrderedDict()

We'll add coordinates and dimension names and see how indexing works:

In [45]:
dims = ('time', 'repetition')
coords = {'time': np.linspace(0, 1, num=10),
          'repetition': np.arange(2)}
da2 = xr.DataArray(np.random.random((10, 2)), dims=dims, coords=coords)
da2

<xarray.DataArray (time: 10, repetition: 2)>
array([[0.206401, 0.691706],
       [0.955573, 0.84368 ],
       [0.566592, 0.021491],
       [0.153732, 0.551681],
       [0.701708, 0.165732],
       [0.028604, 0.407339],
       [0.9873  , 0.018483],
       [0.505805, 0.996356],
       [0.941621, 0.074525],
       [0.825196, 0.275314]])
Coordinates:
  * time        (time) float64 0.0 0.1111 0.2222 0.3333 0.4444 0.5556 0.6667 ...
  * repetition  (repetition) int32 0 1

In [46]:
da2[[0, 1]]  # positional
da2.loc[0.1:0.3, 1]  # rows 1-2 in the second column

<xarray.DataArray (time: 2)>
array([0.84368 , 0.021491])
Coordinates:
  * time        (time) float64 0.1111 0.2222
    repetition  int32 1

In [47]:
da2.isel(time=slice(3, 7))  # dimension name and integer label (sel = select)

<xarray.DataArray (time: 4, repetition: 2)>
array([[0.153732, 0.551681],
       [0.701708, 0.165732],
       [0.028604, 0.407339],
       [0.9873  , 0.018483]])
Coordinates:
  * time        (time) float64 0.3333 0.4444 0.5556 0.6667
  * repetition  (repetition) int32 0 1

In [48]:
da2.sel(repetition=[0])  # dimension name and coordinate label

<xarray.DataArray (time: 10, repetition: 1)>
array([[0.206401],
       [0.955573],
       [0.566592],
       [0.153732],
       [0.701708],
       [0.028604],
       [0.9873  ],
       [0.505805],
       [0.941621],
       [0.825196]])
Coordinates:
  * time        (time) float64 0.0 0.1111 0.2222 0.3333 0.4444 0.5556 0.6667 ...
  * repetition  (repetition) int32 0

Other operations on DataArrays, such as computations, grouping and such, are done very similarly to DataFrames and numpy arrays.

## Dataset

A Dataset is to a DataArray what a DataFrame is to a Series. In other words, it's a collection of DataArrays that share coordinates.

In [49]:
da2  # a reminder. We notice that this could've been a DataFrame as well

<xarray.DataArray (time: 10, repetition: 2)>
array([[0.206401, 0.691706],
       [0.955573, 0.84368 ],
       [0.566592, 0.021491],
       [0.153732, 0.551681],
       [0.701708, 0.165732],
       [0.028604, 0.407339],
       [0.9873  , 0.018483],
       [0.505805, 0.996356],
       [0.941621, 0.074525],
       [0.825196, 0.275314]])
Coordinates:
  * time        (time) float64 0.0 0.1111 0.2222 0.3333 0.4444 0.5556 0.6667 ...
  * repetition  (repetition) int32 0 1

In [50]:
ds = xr.Dataset({'ephys': da2,
                 'calcium': ('time', np.random.random(10))},
                attrs={'AnimalD': 701,
                       'ExperimentType': 'double',
                       'Sex': 'Male'})
ds

<xarray.Dataset>
Dimensions:     (repetition: 2, time: 10)
Coordinates:
  * time        (time) float64 0.0 0.1111 0.2222 0.3333 0.4444 0.5556 0.6667 ...
  * repetition  (repetition) int32 0 1
Data variables:
    ephys       (time, repetition) float64 0.2064 0.6917 0.9556 0.8437 ...
    calcium     (time) float64 0.762 0.935 0.4739 0.8648 0.2908 0.2214 ...
Attributes:
    AnimalD:         701
    ExperimentType:  double
    Sex:             Male

In [51]:
ds['ephys']  # individual DataArrays can be dissimilar in shape

<xarray.DataArray 'ephys' (time: 10, repetition: 2)>
array([[0.206401, 0.691706],
       [0.955573, 0.84368 ],
       [0.566592, 0.021491],
       [0.153732, 0.551681],
       [0.701708, 0.165732],
       [0.028604, 0.407339],
       [0.9873  , 0.018483],
       [0.505805, 0.996356],
       [0.941621, 0.074525],
       [0.825196, 0.275314]])
Coordinates:
  * time        (time) float64 0.0 0.1111 0.2222 0.3333 0.4444 0.5556 0.6667 ...
  * repetition  (repetition) int32 0 1

## Exercise

You're measuring the potential of neurons in a rat's brain over time in response to flashes of light using a multi-electrode array surgically inserted into the rat's skull. Each trial is two seconds long, and one second into the trial a short, 100 ms, bright light is flashed at the animal. After 30 seconds the experiment is replicated, for a total of 4 repetitions. The relevant parameters are the following:
- Rat ID.
- Room temp.
- Room humidity.
- Experimenter name.
- Rat gender.
- Measured voltage (10 electrode, 10k samples representing two seconds).
- Stimulus index (mark differently the pre-, during- and post-stimulus time).
- Repetition number.

Mock data and model it, you can add more parameters if you feel so. You'll use this model for your homework as well, where we will expand and work with it further, so try to build good foundations.

Experimental timeline:
```bob
       1s          100ms             0.9s             30s
Start -----> Flash -----> End flash -----> End trial -----> New trial
|                                                                    |
|--------------------------------------------------------------------|
                                   x4
```

### No exercise solution due to HW assignment